## Get MLB batting stats from 2007 to 2021

In [2]:
import pandas as pd
import csv
from pybaseball import batting_stats

In [3]:
data = batting_stats(2007, 2021, qual=1)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14118 entries, 292 to 1355
Columns: 319 entries, IDfg to xwOBA
dtypes: float64(277), int64(38), object(4)
memory usage: 34.5+ MB


In [6]:
data.to_csv('mlb-batting.csv')